In [1]:
# mlp for multiclass classification
from numpy import argmax
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import tensorflow
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
print(tensorflow.__version__)

2.1.0


Using TensorFlow backend.


In [2]:
data=pd.read_csv("D:/DS/Datasets/iris.csv",encoding='utf-8',header=None,na_values="?")

In [3]:
data.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
X=data.iloc[:,:-1]
X.shape

(150, 4)

In [9]:
y=data.iloc[:,-1:]
y.shape

(150, 1)

In [10]:
# encode strings to integer
y = LabelEncoder().fit_transform(y)

In [11]:
# ensure all data are floating point values
X = X.astype('float32')

In [12]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [9]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(100, 4) (50, 4) (100,) (50,)


In [13]:
# determine the number of input features
n_features = X_train.shape[1]

In [14]:
# fix random seed for reproducibility
#import numpy
#seed = 7
#numpy.random.seed(seed)

In [15]:
def create_model(layers, activation, optimizer='adam', learn_rate=0.01, momentum=0, init_mode='uniform'):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,kernel_initializer=init_mode,input_dim=n_features))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

In [16]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(optimizer=optimizer,layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [17]:
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


c:\users\prade\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\prade\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.740000 using {'activation': 'linear', 'batch_size': 256, 'epochs': 30, 'layers': [40, 20], 'optimizer': 'Nadam'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'SGD'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'RMSprop'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'Adagrad'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'Adadelta'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'Adam'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'Adamax'}
0.320000 (0.054619) with: {'activation': 'softmax', 'batch_size': 128, 'epochs': 30, 'layers': [20], 'optimizer': 'Nadam'}
0.280000 (0.0

In [18]:
[grid_result.best_score_,grid_result.best_params_]

[0.7400000029802323,
 {'activation': 'linear',
  'batch_size': 256,
  'epochs': 30,
  'layers': [40, 20],
  'optimizer': 'Nadam'}]